In [27]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
import re

In [2]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [3]:
rad_data = \
pd.read_sql("""SELECT DISTINCT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 5000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [4]:
rad_data.shape

(5000, 3)

In [7]:
print(rad_data.iloc[20]["text"])




     DATE: [**3352-7-6**] 10:20 AM
     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 1631**]
     Reason: fever- eval for infection
     Admitting Diagnosis: NON-HODGKIN'S LYMPHOMIA\BONE MARROW TRANSPLANT
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      54 year old man with T-cell lymphoma, neutropenia
     REASON FOR THIS EXAMINATION:
      fever- eval for infection
     ______________________________________________________________________________
                                     FINAL REPORT
     INDICATION:  Neutropenia and fever.
     
     COMPARISON: [**6-23**], [**6-6**] and [**3352-6-3**].
     
     CHEST AP PORTABLE RADIOGRAPH:  The cardiac, mediastinal and hilar contours are
     unremarkable.  The left lung is clear.  There is a somewhat well circumscribed
     opacity immediately overlying the catheter which runs along the lateral right
 

### Write a function that returns true or false depending on whether the report contains an impression section

#### Hints

* Not every report will have an impression section
* "INTERPRETATION:" and "CONCLUSIONS:" might be synonyms for "IMPRESSION:"

In [10]:
def count_impression(report): #student solution
    for word in report.split():
        if word == "IMPRESSION:" or word == "INTERPRETATION:" or word == "CONCLUSIONS:":
            return True
    return False

In [ ]:
def count_impression2(report): #Brian's solution is faster
    headings = ["IMPRESSION:", "INTERPRETATION:", "CONCLUSIONS:"]
    for h in headings:
        if h in report:
            return True
    return False

In [11]:
sum([count_impression(report) for report in rad_data["text"]])

4042

### Write a function that returns the impression section of a report



In [13]:
#we assume the impression section is the last section
#if no impression section, return whole report

def get_impression(report):
    headings = ["IMPRESSION:", "INTERPRETATION:", "CONCLUSIONS:"]
    for h in headings:
        if h in report:
            return report[report.find(h):]
    return report

In [15]:
get_impression(rad_data.iloc[20]["text"])

'IMPRESSION:\n     1)  There is a somewhat well-circumscribed but subtle opacity immediately\n     adjacent to the external portion of the catheter overlying the right upper\n     lobe.  This may be related to the catheter itself but it is difficult to\n     exclude an underlying opacity in this region.  Correlation with physical exam\n     recommended.\n\n'

### Define Regular expressions for data cleansing

We have a lot of patterns that are unique and don't convey meaningful information

* De-identified names, dates, etc. 
    * `[**Clip Number (Radiology) 12569**]`
    * `[**Hospital 12568**]`
    * `[**Last Name (NamePattern4) 337**]` 
    * `[**First Name8 (NamePattern2) 12565**]` 
    * `[**Last Name (NamePattern1) 12566**]`
* Separators  (e.g. `__________________`)

### Split into groups and write and test regular expressions to capture these patterns
* Write a regular expression to replace dates in the reports with ``[**DATE**]``
* Write a regular expression to replace times in the reports with ``[**TIME**]``
* Write a regular expression to replaces digits with "d", (e.g. "43 cc" would become "dd cm")

In [16]:
#write a regular expression to capture [**Last Name (NamePattern4) 337**]
last_name=re.compile(r"""\[\*\*Last Name \(NamePattern\d+\) [0-9]*\*\*\]""")



In [18]:
date=re.compile(r"""((?P<month>[A-Z][a-z]{2,}(\.)?) (?P<day>[0-9]{1,2}))""")
last_name=re.compile(r"""(\[\*\*Last Name \((NamePattern|STitle)(\d+)?\) [0-9]*\*\*\])""")
clip = re.compile(r"""\[\*\*Clip Number \(Radiology\) \d+\*\*]""")
date2 = re.compile(r"""DATE: \[\*\*\d+-\d+-\d+\*\*]""")
hospital=re.compile(r"""(\[\*\*Hospital \d+\*\*\*\])""")
unders = re.compile(r"""_{2,}""") #recognize two or more underscores

age2 = re.compile(r"""(?P<age>[0-9]+)(-|\s)y(ear(s)?|\.)(-|\s)?o(ld|\.)""")
age3 = re.compile(r"""\bage(d)? (?P<age>[0-9]+)""")
digits = re.compile(r"""\d""")
def age_in_decades(m):
    age = int(m.group("age"))
    
    return "[** Age in %ss**]"%(int(age/10)*10,)

age_in_decades(next(age2.finditer("74-year-old")))
#tmp = re.sub(age2, age_in_decades, re.sub(age3, age_in_decades, report_txt))

'[** Age in 70s**]'

#### Hints

* Look at some sample reports to see what dates and times look like in the reports
* What order would you need to apply the regular expressions?

In [33]:
def preprocess(report):
    return digits.sub("d",
                      unders.sub("\n", 
                                 hospital.sub("HOSPITAL", 
                                              date2.sub("DATE", 
                                                        clip.sub("CLIP", 
                                                                 last_name.sub("LASTNAME", report))))))

In [34]:
for i in range(10):
    print(preprocess(rad_data.iloc[i]['text']))
    print("*"*42,"\n\n")




     DATE dd:dd AM
     MR HEAD W & W/O CONTRAST; MR CONTRAST GADOLIN                   Clip # CLIP
     Reason: R ICB and HX brain mets - eval - also with DWI for CVA Do MR
      Contrast: MAGNEVIST Amt: dd
     

     UNDERLYING MEDICAL CONDITION:
      [**Age over dd **] year old woman with lung CA- mets to brain                                   
     REASON FOR THIS EXAMINATION:
      R ICB and HX brain mets - eval - also with DWI for CVA Do MRI both with and 
      without contast please
     

                                     FINAL REPORT
     EXAMINATION:  MRI of the brain with and without gadolinium.
     
     INDICATION:  [**Age over dd **] year old woman with lung cancer and right intracranial bleed
     and history of brain metastases.  Please evaluate for acute infarct.
     
     TECHNIQUE:  Multiplanar Td and Td-weighted images of the brain with gadolinium
     according to standard departmental protocol.  No prior study for comparison.
     
     FINDINGS:  On d

### You can use the Pandas `iloc` method to grab specific reports

In [ ]:
print(rad_data.iloc[0]["text"])

In [ ]:
print(rad_data.iloc[0]["text"])

### Create a new column named "impression" for storing just the impression portion of the report

In [21]:
#take our impression sections, clean them up, and put into the dataframe as a new column
rad_data["impression"] = \
rad_data.apply(lambda row: preprocess(get_impression(row["text"])).lower(), axis=1)

In [22]:
rad_data.head()

,subject_id,hadm_id,text,impression
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date 10:53 am\n mr head w & w/o...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,\n\n\n date 10:53 am\n mr head w & w/o...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,impression: stable appearance of right pariet...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,impression:\n \n cardiomegaly and mild...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,impression:\n \n marked improvement in...


In [ ]:
print(rad_data.iloc[20]["impression"])

### How many unique words occur in the corpus?

#### Hint

1. Use TextBlob
1. Put all the reports into a single string

#### I got 8658

In [ ]:
#" ".join(rad_data["impression"])

In [37]:
#!python -m textblob.download_corpora
blob = TextBlob(" ".join(rad_data["impression"]))
unique_impression_words = set(blob.words)
len(unique_impression_words)

#pass TextBlob a string and it gets the words for us
#" " is the delimiter for join -- so we make one big string with empty string between
#set for uniqueness

#Brian gets 7402 and apparently he is right and we are wrong

8485

In [36]:
unique_impression_words

{'centra',
 'ea',
 '2915-9-2',
 'pain',
 '78-year-odl',
 'pronounced',
 'handed',
 '2:11',
 't12',
 '9:28',
 'should',
 'regard',
 'symmetry',
 'lefet',
 'caring',
 '1947',
 'top',
 'hemidiaphgram',
 '143',
 'bradycardia',
 'panniculitis',
 '2271',
 'does',
 'underperfusion',
 'nondependent',
 'mmp',
 'lesion',
 'towards',
 'results',
 'cxrs',
 'barely',
 'rhonchi',
 'jp',
 'soluble',
 'ear',
 'created',
 'would',
 'outlined',
 'undergoing',
 '225',
 'endograft',
 'rewire',
 'consult',
 'detail',
 'pseudoaneurysmal',
 'overload',
 'supratentorial',
 '3350-10-18',
 'cub',
 '8:10',
 'recto-sigmoid',
 '2847-6-29',
 '172',
 'tolerating',
 '10292',
 'obliterates',
 'leftclip',
 'coming',
 'represents',
 '12:24',
 'lacunar',
 '87',
 'lobar',
 'patient',
 'moderate',
 'ruq',
 'flowing',
 'story',
 '2617-10-24',
 'overall',
 '2962-10-19',
 '2798-11-3',
 'optimal',
 'bp',
 'acetabular',
 'cardiopulm',
 'post-tpa',
 'hyper-echoic',
 'intr',
 '442',
 'pulm',
 'elastic',
 '6:16',
 'intraperitoneal

In [39]:
from collections import Counter

c = Counter(blob.words)
c.most_common()

[('the', 15148),
 ('of', 9815),
 ('and', 5570),
 ('is', 4910),
 ('with', 4789),
 ('in', 4376),
 ('impression', 4153),
 ('right', 3518),
 ('to', 3401),
 ('no', 3237),
 ('left', 2927),
 ('clip', 2794),
 ('for', 2643),
 ('a', 2599),
 ('reason', 2189),
 ('there', 2057),
 ('contrast', 1982),
 ('chest', 1970),
 ('ct', 1902),
 ('1', 1754),
 ('this', 1723),
 ('2', 1596),
 ('date', 1555),
 ('report', 1533),
 ('tube', 1473),
 ('final', 1473),
 ('at', 1460),
 ('was', 1427),
 ('pleural', 1403),
 ('are', 1374),
 ('on', 1357),
 ('evidence', 1277),
 ('or', 1230),
 ('small', 1186),
 ('placement', 1070),
 ('artery', 1040),
 ('be', 1013),
 ('examination', 976),
 ('3', 954),
 ('bilateral', 930),
 ('as', 906),
 ('effusion', 905),
 ('pm', 903),
 ('within', 897),
 ('catheter', 885),
 ('lower', 882),
 ('over', 879),
 ('diagnosis', 879),
 ('lobe', 864),
 ('pulmonary', 863),
 ('pneumothorax', 854),
 ('interval', 852),
 ('admitting', 824),
 ('underlying', 821),
 ('study', 817),
 ('ap', 815),
 ('line', 809),
 ('

In [40]:
#essentially meaningless words are called stopwords
#usually remove these before you continue with creating vectors

from gensim.parsing.preprocessing import STOPWORDS
STOPWORDS
#gensim is not specific to medical text...
#it removes no, which is bad for medical text e.g. "no [bad thing]"
#can't use a generic stopword list

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [41]:
my_stop_words = frozenset(["a", "am", "an", "and", "are", "as", "at", "be", "for", "is", "the", "is", "of", "which", ])

## Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame

### Define a vector space for the radiology corpus

#### Hints

1. How would you build a corpus from words only occuring more than N times?

### Create a new column named `"impression no stops"` where [stop words](https://en.wikipedia.org/wiki/Stop_words) have been dropped from the impression

* The gensim package has stop words defined (``from gensim.parsing.preprocessing import STOPWORDS``

#### Hints
1. Do you agree with dropping all the stop words?
1. How could we create a new stopwords frozen set absent the terms we wan't to keep (double negative?)
1. You could use a regular expressions substitution or token the report first and operate on the list of words.

In [42]:
rad_data["impression no stops"] = \
rad_data.apply(lambda row: \
               " ".join([word for word in row["impression"].split() if word not in my_stop_words]), axis=1)

In [43]:
rad_data.iloc[0]["impression"]

"\n\n\n     date 10:53 am\n     mr head w & w/o contrast; mr contrast gadolin                   clip # clip\n     reason: r icb and hx brain mets - eval - also with dwi for cva do mr\n      contrast: magnevist amt: 15\n     \n\n     underlying medical condition:\n      [**age over 90 **] year old woman with lung ca- mets to brain                                   \n     reason for this examination:\n      r icb and hx brain mets - eval - also with dwi for cva do mri both with and \n      without contast please\n     \n\n                                     final report\n     examination:  mri of the brain with and without gadolinium.\n     \n     indication:  [**age over 90 **] year old woman with lung cancer and right intracranial bleed\n     and history of brain metastases.  please evaluate for acute infarct.\n     \n     technique:  multiplanar t1 and t2-weighted images of the brain with gadolinium\n     according to standard departmental protocol.  no prior study for comparison.\n 

In [44]:
rad_data.iloc[0]["impression no stops"]

"date 10:53 mr head w & w/o contrast; mr contrast gadolin clip # clip reason: r icb hx brain mets - eval - also with dwi cva do mr contrast: magnevist amt: 15 underlying medical condition: [**age over 90 **] year old woman with lung ca- mets to brain reason this examination: r icb hx brain mets - eval - also with dwi cva do mri both with without contast please final report examination: mri brain with without gadolinium. indication: [**age over 90 **] year old woman with lung cancer right intracranial bleed history brain metastases. please evaluate acute infarct. technique: multiplanar t1 t2-weighted images brain with gadolinium according to standard departmental protocol. no prior study comparison. findings: on diffusion-weighted images there small area restricted diffusion along falx within left occipitotemporal lobe. it also bright on flair-weighted images may represent subacute infarct. clinical correlation recommended. on gradient echo images there large area intraparenchymal hemor

### What are the unique words in our vocabulary?

In [46]:
unique_impression_words = set(TextBlob(" ".join(rad_data["impression no stops"])).words)


### We'll create a vocabulary with `zip` and `dict`

In [47]:
word_map = dict(zip(unique_impression_words,range(len(unique_impression_words))))

In [48]:
len(word_map)

8479

In [49]:
word_map

{'75625': 743,
 'centra': 0,
 'variant': 7028,
 'ea': 1,
 'silhouette': 4267,
 'oral': 4268,
 'pseudo-obstruction': 4269,
 'perforated': 8187,
 'workup': 5650,
 '2915-9-2': 2,
 'pain': 3,
 'part': 4271,
 '78-year-odl': 4,
 'sachs': 4273,
 'tomorrow': 4274,
 'regarding': 7763,
 'pronounced': 5,
 'handed': 6,
 't12': 8,
 '3:52': 4276,
 'flouroscopic': 4277,
 '9:28': 9,
 'should': 10,
 'regard': 11,
 'stones/sludge/cbd': 5558,
 'symmetry': 12,
 'unfolded': 4278,
 'lefet': 13,
 '141': 4279,
 'fluctuating': 4280,
 'caring': 14,
 '2558-5-25': 4281,
 '1947': 15,
 'top': 16,
 '143': 18,
 'making': 7765,
 'abnormally': 4283,
 'bradycardia': 19,
 'porta': 4270,
 'panniculitis': 20,
 '258': 4287,
 '2271': 21,
 'does': 22,
 'underperfusion': 23,
 'nondependent': 24,
 'thrombosis': 3536,
 'mmp': 25,
 'lesion': 26,
 'towards': 27,
 'ipf': 4288,
 'cxrs': 29,
 'corresponding': 4289,
 'west': 2756,
 'jp': 32,
 'soluble': 33,
 'ct': 4293,
 'marker': 7032,
 'transfixing': 1218,
 'february': 4272,
 '2-5':

### Save for use in our next notebook

In [50]:
import gzip
import pickle

with open("rad_data.pickle.gz", "wb") as f0:
    pickle.dump(rad_data, f0)

In [51]:
with open("rad_vocabulary.pickle.gz", "wb") as f0:
    pickle.dump(word_map, f0)